## Data Preparation

This notebooks joins data, aligns time granularity and does some initial feature engineering for clustering.

In [1]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

In [2]:
cwd = os.getcwd()
DATA_DIR = os.path.join(cwd, '..', 'data')
INT_DATA_DIR = os.path.join(cwd, '..', 'data', 'interim')
RAW_DATA_DIR = os.path.join(cwd, '..', 'data', 'raw')

In [3]:
def load_data():
    """Load all required CSV files"""
    consumptions = pd.read_csv(os.path.join(RAW_DATA_DIR,'customers_consumptions.csv'))
    metadata = pd.read_csv(os.path.join(RAW_DATA_DIR,'customers_metadata.csv'))
    weather = pd.read_csv(os.path.join(RAW_DATA_DIR,'weather_data.csv'))
    prices = pd.read_csv(os.path.join(RAW_DATA_DIR,'price_data.csv'))
    
    # Convert timestamp columns to datetime
    consumptions['validfrom'] = pd.to_datetime(consumptions['validfrom'])
    weather['validfrom'] = pd.to_datetime(weather['validfrom'])
    prices['validfrom'] = pd.to_datetime(prices['validfrom'])
    
    # sort
    consumptions = consumptions.sort_values(by=["meteringpoint_id", "validfrom"])
    weather = weather.sort_values(by=["weatherstation_id", "validfrom"])
    
    return consumptions, metadata, weather, prices

In [4]:
def resample_and_ffill_prices(df, timestamp_col, freq='15T', value_cols=None):
    """
    Resample a DataFrame to a specified frequency and forward fill values.
    """
    df = df.copy()
    df[timestamp_col] = pd.to_datetime(df[timestamp_col])
    if value_cols is None:
        value_cols = [col for col in df.columns if col != timestamp_col]
    df = df.set_index(timestamp_col)
    df_resampled = df[value_cols].resample(freq).ffill()
    df_resampled = df_resampled.reset_index()
    
    return df_resampled

In [5]:
def fill_weather_gaps(weather_df, freq='15T'):
    """
    Fill gaps in weather data by resampling and forward filling.
    """
    df = weather_df.copy()
    
    # Ensure timestamp is datetime
    df['validfrom'] = pd.to_datetime(df['validfrom'])
    value_cols = ['air_temp', 'ghi', 'cloud_opacity', 'precipitable_water']
    
    filled_dfs = []
    for station_id in df['weatherstation_id'].unique():
        station_data = df[df['weatherstation_id'] == station_id].copy()
        station_data = station_data.set_index('validfrom')
        filled_data = station_data[value_cols].resample(freq).ffill()
        filled_data = filled_data.reset_index()
        filled_data['weatherstation_id'] = station_id
        
        filled_dfs.append(filled_data)
    
    result = pd.concat(filled_dfs, ignore_index=True)
    result = result.sort_values(['weatherstation_id', 'validfrom'])
    
    return result

In [ ]:
def resample_consumption(df, freq='1H'):
    """
    Resample consumption data to a specified frequency and aggregate values.
    """
    df = df.copy()
    
    # Ensure timestamp is datetime
    df['validfrom'] = pd.to_datetime(df['validfrom'])
    
    filled_dfs = []
    
    # Process each meter separately
    for meter_id in df['meteringpoint_id'].unique():
        meter_data = df[df['meteringpoint_id'] == meter_id].copy()
        meter_data = meter_data.set_index('validfrom')
        filled_data = meter_data[['quantity']].resample(freq).mean()
        filled_data = filled_data.reset_index()
        filled_data['meteringpoint_id'] = meter_id
        
        filled_dfs.append(filled_data)
    
    result = pd.concat(filled_dfs, ignore_index=True)
    result = result.sort_values(['meteringpoint_id', 'validfrom'])
    
    print(f"Original shape: {df.shape}")
    print(f"Resampled shape: {result.shape}")
    
    return result

In [6]:
def normalise(df, group_col='meteringpoint_id'):
    """
    Normalize data using z-score within each group.
    """
    normalized = df.copy()
    
    # Columns to normalize
    cols_to_normalize = ['quantity', 'air_temp', 'ghi', 'cloud_opacity', 
                        'precipitable_water', 'price']
    
    # Normalize each feature within each group
    for col in cols_to_normalize:
        normalized[f'{col}_norm'] = (df.groupby(group_col)[col]
                                   .transform(lambda x: (x - x.mean()) / x.std()))
    
    return normalized

In [7]:
consumptions_df, metadata_df, weather_df, prices_df = load_data()

In [96]:
weather_df = fill_weather_gaps(weather_df, freq='15T')

In [97]:
prices_df = resample_and_ffill_prices(
    df=prices_df,
    timestamp_col='validfrom',
    freq='15T',
    value_cols=['price']
)

In [98]:
merged_df = (consumptions_df
 .merge(metadata_df, on=["meteringpoint_id"], how='left')
 .merge(weather_df, on=["weatherstation_id", "validfrom"], how='left')
 .merge(prices_df, on=["validfrom"], how="left")
 .sort_values(by=["meteringpoint_id", "validfrom"])
 .drop_duplicates())

In [99]:
normal_df = normalise(merged_df)

In [8]:
consumptions_df

,meteringpoint_id,validfrom,quantity
4,261b7930-dd7e-43f0-8a42-4f31d97f9edb,2024-01-01 00:00:00+00:00,0.156
14,261b7930-dd7e-43f0-8a42-4f31d97f9edb,2024-01-01 00:15:00+00:00,0.306
22,261b7930-dd7e-43f0-8a42-4f31d97f9edb,2024-01-01 00:30:00+00:00,0.211
28,261b7930-dd7e-43f0-8a42-4f31d97f9edb,2024-01-01 00:45:00+00:00,0.184
36,261b7930-dd7e-43f0-8a42-4f31d97f9edb,2024-01-01 01:00:00+00:00,0.523
...,...,...,...
444067,eba9a77e-589f-4c63-b7d2-8506c0cb8a59,2024-10-31 22:45:00+00:00,0.002
444089,eba9a77e-589f-4c63-b7d2-8506c0cb8a59,2024-10-31 23:00:00+00:00,0.001
444108,eba9a77e-589f-4c63-b7d2-8506c0cb8a59,2024-10-31 23:15:00+00:00,0.002
444134,eba9a77e-589f-4c63-b7d2-8506c0cb8a59,2024-10-31 23:30:00+00:00,0.001


## Saving data

In [100]:
merged_df.to_csv(os.path.join(INT_DATA_DIR, 'merged.csv'), index=False)

In [101]:
normal_df.to_csv(os.path.join(INT_DATA_DIR, 'norm_merged.csv'), index=False)